In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel

In [3]:
def contraction(params,x,p):
    #beta and x are kind of parameters. x is the empirical distribution of x?
    k = int(x.shape[1]/2)
    util1 = np.dot(x[:,0:k], params[0:k])  + params[k]*p[0]
    util2 = np.dot(x[:,k:], params[k+1:2*k+1]) + params[2*k+1]*p[1]
    contr_result = [np.exp(util1)/(1+np.exp(util1)),np.exp(util2)/(1+np.exp(util2))]
    return np.array(contr_result)

#actually caclualte an equilibrium in this game
N=1000
ps = np.array([.5,.5])
betas = np.array([1,-2, 2,-1])

#set up xs
xs = np.random.normal(scale=2, size=(N,2)) #assumes xs are independent? could use a copula?
xs = pd.DataFrame(xs,columns = ['x11','x12'])
xs = np.array(xs)

print(contraction(betas,xs,ps))

[[0.31669303 0.5452179  0.69628106 ... 0.09177064 0.00247028 0.24540238]
 [0.00969015 0.00281613 0.99624504 ... 0.00263024 0.03570123 0.20279688]]


In [4]:
def contraction_map(betas,x,p):
    """final result is beliefs of firm1/firm2"""
    for i in range(50):
        #print('b1: %.4f, p2: %.4f'%tuple(p))
        p = contraction(betas,x,p).mean(axis=1)
        p = np.flip(p)
        #print('p1: %.4f, p2: %.4f'%tuple(p))
        #print('----- end of iteration %s ----'%i)
    return p

result_ps = contraction_map(betas,xs,ps)
us = np.random.logistic(size=(1000,2))

k = int(xs.shape[1]/2)
es = np.random.logistic(0, 1, (2,N) )
y1 = 1*(np.dot(xs[:,0:k], betas[0:k])  + betas[k]*result_ps[0] + es[0,:] >= 0)
y2 = 1*(np.dot(xs[:,k:], betas[k+1:2*k+1]) + betas[2*k+1]*result_ps[1] + es[1,:] >= 0)
ys = np.array([y1,y2]).transpose()
print(np.flip(result_ps))
print(ys.mean(axis=0))

[0.35369953 0.47480514]
[0.37  0.476]


In [5]:
result_df = np.concatenate( (ys,xs ) ,axis=1)
result_df = pd.DataFrame(result_df, columns=['y1','y2','x11','x12'])
print(result_df)
result_df.to_csv('monte_carlo.csv')

      y1   y2       x11       x12
0    1.0  0.0  0.230989 -2.063454
1    1.0  0.0  1.181367 -2.684785
2    1.0  1.0  1.829651  3.040458
3    0.0  1.0 -3.131189  1.150653
4    0.0  0.0 -2.193524  2.269287
..   ...  ...       ...       ...
995  0.0  1.0 -1.184860  1.417917
996  0.0  1.0 -1.566161  3.532050
997  0.0  0.0 -1.292205 -2.719023
998  0.0  0.0 -5.000950 -1.398108
999  0.0  1.0 -0.123285 -0.434452

[1000 rows x 4 columns]


In [6]:
class BayesNashLogit(GenericLikelihoodModel):
    
    def nloglikeobs(self, params):
        n = self.exog.shape[0]
        k = int(self.exog.shape[1]/2)
        
        p = self.endog.mean(axis=0)
        p = contraction_map(params,self.exog,p)
        
        likelihood = contraction(params,self.exog,p).transpose()
        ll = self.endog*np.log(likelihood) + (1-self.endog)*np.log(1-likelihood)
        
        return -1*ll.sum()
        
        
    
    def fit(self, **kwds):
        """fit the likelihood function using the right start parameters"""
        start_params = np.ones(self.exog.shape[1]+2)
        return super(BayesNashLogit, self).fit(start_params=start_params,**kwds)

        
        
N = result_df.shape[0]
ys = result_df[['y1','y2']]
xs = result_df[['x11' ,'x12']]


model = BayesNashLogit(ys,xs)
model_fit = model.fit(xtol=1e-12,ftol=1e-12)
print(model_fit.summary(xname=["b11","d1","b12","d2"]))

Optimization terminated successfully.
         Current function value: 0.734661
         Iterations: 233
         Function evaluations: 477
                            BayesNashLogit Results                            
Dep. Variable:           ['y1', 'y2']   Log-Likelihood:                -734.66
Model:                 BayesNashLogit   AIC:                             1473.
Method:            Maximum Likelihood   BIC:                             1483.
Date:                Mon, 19 Apr 2021                                         
Time:                        10:08:09                                         
No. Observations:                1000                                         
Df Residuals:                     998                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

In [7]:
#attempt at standard errors...
H = np.linalg.inv(model.hessian(model_fit.params)/N)
diag = np.diagonal(np.linalg.inv(model.hessian(model_fit.params)))
print(np.sqrt(-1*diag))

[0.06558707 0.18809362 0.13504876 0.29493431]


In [8]:
# Modify the variables
entry_data = pd.read_csv("merged_entry.csv")
entry_data.loc[entry_data['HD'] > 0.0, 'HD_1'] = 1
entry_data.loc[entry_data['LO'] > 0.0, 'LO_1'] = 1
entry_data['income_2'] = np.log(1 + entry_data['income_per_capita'])
entry_data['population_2'] = np.log(1 + entry_data['population'])
entry_data['under44_1_2'] = np.log(1 + entry_data['under44_1'])
entry_data['under44_2_2'] = np.log(1 + entry_data['under44_2'])
entry_data['under44_3_2'] = np.log(1 + entry_data['under44_3'])
entry_data['older65_1_2'] = np.log(1 + entry_data['older65_1'])
entry_data['older65_2_2'] = np.log(1 + entry_data['older_65_2'])
entry_data = entry_data.fillna(0)

C:\Users\himan\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [11]:
# Fit the model
N = entry_data.shape[0]
x = entry_data[['income_2','population_2','income_2','population_2']].copy()
y = entry_data[['HD_1','LO_1']]

model = BayesNashLogit(y,x).fit(xtol=1e-6,ftol=1e-6)
model.summary()

C:\Users\himan\anaconda3\lib\site-packages\statsmodels\base\model.py:567: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warn("Maximum Likelihood optimization failed to converge. "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            BayesNashLogit Results                            
==============================================================================
Dep. Variable:       ['HD_1', 'LO_1']   Log-Likelihood:                -2730.7
Model:                 BayesNashLogit   AIC:                             5465.
Method:            Maximum Likelihood   BIC:                             5477.
Date:                Mon, 19 Apr 2021                                         
Time:                        10:09:57                                         
No. Observations:                2155                                         
Df Residuals:                    2153                                         
Df Model:                           1                                         
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
income_2        -0.2697      0.040     -6.752      0.000      -0.348      -0.191
population_2     0.2743      0.038      7.155      0.000       0.199       0.350
income_2         2.1167      0.301      7.033      0.000       1.527       2.707
population_2    -0.2629      0.037     -7.144      0.000      -0.335      -0.191
par0             0.1446      0.034      4.286      0.000       0.078       0.211
par1             1.6739      0.220      7.614      0.000       1.243       2.105
================================================================================
"""